<a href="https://colab.research.google.com/github/KafeisM/aa_24/blob/main/vgg_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/KafeisM/aa_24/blob/main/vgg_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align: center;">
<a target="_blank" href="https://colab.research.google.com/github/bmalcover/aa_2425/blob/main/10_VGG_Custom_Dataset/VGG_Custom.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
</div>

In [1]:
import os

import torch
import torch.optim as optim
from tqdm.auto import tqdm

from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET

import cv2

import glob

# Introducció

En aquesta pràctica, treballarem amb un conjunt de dades d'imatges de cans i moixos que ja heu utilitzat a la pràctica de la part anterior de l'assignatura. Tot i que coneixem aquest conjunt de dades, en aquesta ocasió ho em d'emprar amb ``Pytorch`` per tant haurem d'adaptar com ho llegim: fent un pre-processat de les dades per aplicar-los nous models d'aprenentatge profund.

L'objectiu de la sessió serà experimentar amb les diferents versions de la xarxa de convolució ``VGG``, com ara VGG16 i VGG19. Aquesta pràctica us permetrà veure com canvien els resultats d'entrenament i predicció en funció de l'arquitectura emprada, i reflexionar sobre els avantatges i inconvenients d'incrementar la profunditat d'una xarxa de convolució.

# Preparam les dades
Primerament descarregam les dades en el Google Colab. Per tal de fer-ho emprarem les eines ``wget`` i ``unzip``.


In [ ]:
!wget https://github.com/bmalcover/aa_2425/releases/download/v1/gatigos.zip
!unzip gatigos.zip

# *Custom dataset*

Un dataset personalitzat en PyTorch permet preparar i gestionar dades específiques per a un projecte d’aprenentatge automàtic. La classe ``Dataset`` de ``PyTorch`` és la base per crear aquest tipus de dataset i requereix la implementació de tres mètodes essencials: ``__init__``, ``__len__``, i ``__getitem__``.

* ``__init__``: Aquest mètode inicialitza el ``dataset`` i defineix els paràmetres que seran necessaris, com ara la ubicació de les dades o qualsevol transformació a aplicar. Aquí es poden carregar rutes d'imatges o etiquetes i definir les transformacions que es realitzaran.

* ``__len__``: Aquest mètode retorna el nombre total d'exemples en el ``dataset``. ``PyTorch`` l'utilitza per saber quantes mostres conté el conjunt de dades, cosa que és essencial per crear les batchs d’entrenament.
* ``__getitem__``: Aquest mètode accedeix a una mostra concreta del ``dataset``, identificada per un índex, i retorna les dades i la seva etiqueta (o ``target``). Normalment, s’apliquen les transformacions aquí abans de retornar la mostra, per assegurar que cada dada té el format adequat per al model.

Un cop creada, aquesta classe es pot emprar amb el ``DataLoader`` de ``PyTorch`` per gestionar l'entrenament en *batchs*, fent que el dataset personalitzat sigui eficient i fàcil de treballar dins del flux d’aprenentatge automàtic de ``PyTorch``.


In [6]:
class CatIGosDataset(Dataset):
    """Cat i Gos dataset."""

    def __init__(self, img_path, annotations, transform=None):
        """

        Args:
            img_path (str): Ruta a la carpeta d'imatges
            annotations (dict): Diccionari amb les anotacions de les imatges.
            transform (callable, optional): Transformacions a aplicar a les imatges.
        """
        self.pathIMG = img_path
        self.labels = annotations
        self.transform = transform


    def __len__(self):
        return len(self.pathIMG)

    def __getitem__(self, idx):
        img = cv2.imread(self.pathIMG[idx])
        if img is None:
            raise ValueError(f"Image not found or unable to read: {self.pathIMG[idx]}")

        if len(img.shape) == 2 or img.shape[2] == 1:
            img = cv2.cvtColor(img, cv2.COLOR_BAYER_BG2BGR)
        elif img.shape[2] == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        else:
            raise ValueError(f"Unexpected number of channels in image: {self.pathIMG[idx]}")

        label = self.labels[self.pathIMG[idx]]

        if self.transform:
            img = self.transform(img)

        return img, label


In [12]:
!ls /content/
paths_imgs = glob.glob("/content/images/*.png")
labels = {}

for path_img in paths_imgs:
  _,img_name = os.path.split(path_img)
  name = img_name.split(".")[0]

  name_xml = f"/content/annotations/{name}.xml"
  tree = ET.parse(name_xml)
  root = tree.getroot()

  label = root.find("object").find("name").text
  annotation = 0 if label == 'cat' else 1

  labels[path_img] = annotation


annotations  gatigos.zip  images  sample_data


In [13]:
BATCH_SIZE = 4
EPOCHS = 5

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

print(len(paths_imgs))
print(len(labels))
DataSet = CatIGosDataset(paths_imgs,labels,transform=transform)

train_dataset, test_dataset = train_test_split(DataSet,test_size=0.33,random_state=42)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


3686
3686


# Definicio de la xarxa: VGG i *Transfer learning*

En aquesta pràctica aplicarem la tècnica de transfer learning amb una de les xarxes CNN més conegudes i profundes:

 - VGG. [Very Deep Convolutional Networks for Large-Scale Image Recognition, 2014](https://arxiv.org/abs/1409.1556). La mida d'entrada de les imatges és de (224x224x3). VGG es presenta en diferents variants, com ara VGG16 i VGG19, que contenen respectivament 16 i 19 capes amb aproximadament 138 milions de paràmetres entrenables en el cas de VGG16.

Descarregarem VGG i l'analitzarem. En aquest cas, no només obtenim la seva arquitectura, sinó també els pesos resultants del seu entrenament en grans conjunts de dades.





In [15]:
vgg11 = models.vgg11(weights=True)

print("-" * 50)
print("Arquitectura VGG11")
print("-" * 50)
print(vgg11)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth
100%|██████████| 507M/507M [00:06<00:00, 84.0MB/s]


--------------------------------------------------
Arquitectura VGG11
--------------------------------------------------
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1

## Com emprar la GPU per entrenar un model

Un dels elements diferencials d'aquest model, respecte als que havíem vist fins ara, és la seva mida i, per tant, l'entrenament es torna impossible emprant __CPU__ directament. Per resoldre-ho hem d'emprar una **GPU**, a Google Colab disposam d'elles gratuïtament. Per fer-ho amb *Pytorch* hem de fer tres passes:

1. Comprovar que hi ha una GPU disponible.
2. Moure el model a GPU.
3. Moure les dades a GPU.

### Comprova si tenim una GPU disponible

Primer de tot, cal verificar si hi ha una GPU disponible a l’entorn. Això es pot fer amb el següent codi:

```python

import torch

is_cuda = torch.cuda.is_available()
```

Si la variable ``is_cuda`` és certa, llavors tens accés a una GPU.

### Mou el model a la GPU

En PyTorch, els models han d'estar explícitament en la GPU per poder fer servir la seva potència de càlcul. Si estàs carregant un model preentrenat (com AlexNet, ResNet, etc.), o si has definit el teu propi model, pots moure’l a la GPU amb ``.to(device)``, on device fa referència a la GPU.

```python

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
```

Això mou el model a la GPU (si està disponible). Si només tens una CPU, el model es mantindrà a la CPU.

### Mou les dades a la GPU

No només el model, sinó que també les dades (inputs) han d'estar a la GPU per fer les operacions més ràpides. Així, abans de fer servir les dades com a inputs del model, assegura't de moure-les al mateix device:

```python

# Exemple d'un batch de dades
inputs, labels = inputs.to(device), labels.to(device)
```


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
vgg11.to(device)

cuda


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

# Entrenament

In [18]:
# prompt: dame el codigo del entremiento del modelo haciendo uso der transfer learning

# Freeze all layers except the last few
for param in vgg11.features.parameters():
    param.requires_grad = False

# Replace the classifier with a new one for binary classification
num_features = vgg11.classifier[6].in_features
vgg11.classifier[6] = torch.nn.Linear(num_features, 2) # 2 output classes

# Move the model to the device (GPU if available)
vgg11.to(device)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg11.classifier[6].parameters(), lr=0.001, momentum=0.9)

# Training loop
for epoch in range(EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg11(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 2000 == 1999:  # Print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Epoch 1/5:   0%|          | 0/618 [00:00<?, ?it/s]

Epoch 2/5:   0%|          | 0/618 [00:00<?, ?it/s]

Epoch 3/5:   0%|          | 0/618 [00:00<?, ?it/s]

Epoch 4/5:   0%|          | 0/618 [00:00<?, ?it/s]

Epoch 5/5:   0%|          | 0/618 [00:00<?, ?it/s]

Finished Training



## Feina a fer:

1. Preparar el *dataset* personalitzat.
2. Carregar la xarxa VGG11, VGG16 i VGG19, amb i sense batch normalization.
3. Entrenar-ho fent *transfer learning*.
4. Comparar els resultats.


In [19]:
# prompt: enseñame los resultados del entrenamiento haciendo un test del modelo vgg11

# Testing loop
correct = 0
total = 0
with torch.no_grad():
    for data in tqdm(test_loader, desc="Testing"):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = vgg11(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

Testing:   0%|          | 0/618 [00:00<?, ?it/s]

Accuracy of the network on the test images: 99 %
